##Name- CS Venkata Adithya
##Pin Number-221910301015

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [3]:
from google.colab import files
uploaded = files.upload()

Saving franklin.txt to franklin (1).txt


In [4]:
file = open("franklin.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]

"Project Gutenberg's Autobiography of Benjamin Franklin, by Benjamin Franklin This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org Title: Autobiography of Benjamin Franklin Author: Benjamin Franklin Editor: Frank Woodworth Pine Illustrator: E. Boyd Smith Release Date: December 28, 2006 [EBook #20203] [Las"

In [5]:
len(data)

451189

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[84, 4543, 238, 2, 335, 87, 19, 335, 87, 26, 843, 40, 12, 1, 138]

In [7]:
len(sequence_data)

79803

In [8]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8307


In [9]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  79800


array([[  84, 4543,  238,    2],
       [4543,  238,    2,  335],
       [ 238,    2,  335,   87],
       [   2,  335,   87,   19],
       [ 335,   87,   19,  335],
       [  87,   19,  335,   87],
       [  19,  335,   87,   26],
       [ 335,   87,   26,  843],
       [  87,   26,  843,   40],
       [  26,  843,   40,   12]])

In [10]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [11]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[  84 4543  238]
 [4543  238    2]
 [ 238    2  335]
 [   2  335   87]
 [ 335   87   19]
 [  87   19  335]
 [  19  335   87]
 [ 335   87   26]
 [  87   26  843]
 [  26  843   40]]
Response:  [  2 335  87  19 335  87  26 843  40  12]


In [12]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [14]:
model.compile(loss="categorical_crossentropy",metrics=['accuracy'], optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64)

Epoch 1/70
1247/1247 [==============================] - 28s 17ms/step - loss: 6.7106 - accuracy: 0.0600
Epoch 2/70
1247/1247 [==============================] - 21s 17ms/step - loss: 6.2183 - accuracy: 0.0898
Epoch 3/70
1247/1247 [==============================] - 22s 18ms/step - loss: 5.8917 - accuracy: 0.1088
Epoch 4/70
1247/1247 [==============================] - 21s 17ms/step - loss: 5.6160 - accuracy: 0.1232
Epoch 5/70
1247/1247 [==============================] - 21s 17ms/step - loss: 5.3777 - accuracy: 0.1343
Epoch 6/70
1247/1247 [==============================] - 21s 17ms/step - loss: 5.1583 - accuracy: 0.1440
Epoch 7/70
1247/1247 [==============================] - 21s 17ms/step - loss: 4.9278 - accuracy: 0.1544
Epoch 8/70
1247/1247 [==============================] - 21s 17ms/step - loss: 4.6739 - accuracy: 0.1631
Epoch 9/70
1247/1247 [==============================] - 21s 17ms/step - loss: 4.3993 - accuracy: 0.1746
Epoch 10/70
1247/1247 [==============================] - 21s 17m

In [15]:
model.save('next_words.h5')

In [16]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [19]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: Project Gutenberg's
['Project', "Gutenberg's"]
1/1 [==============================] - 0s 19ms/step
to
Enter your line: Project Gutenberg's Autobiography
['Project', "Gutenberg's", 'Autobiography']
1/1 [==============================] - 0s 17ms/step
of
Enter your line: This eBook is for the use of anyone
['use', 'of', 'anyone']
1/1 [==============================] - 0s 19ms/step
anywhere
Enter your line: Produced by Turgut Dincer,
['by', 'Turgut', 'Dincer,']
1/1 [==============================] - 0s 16ms/step
and
Enter your line: In the simplicity and
['the', 'simplicity', 'and']
1/1 [==============================] - 0s 20ms/step
vigor
Enter your line: 0
Execution completed.....
